## Tool creation
* Using the @tool decorator for simple function-based tools
* Creating a subclass of Tool for more complex functionality

In [ ]:
from smolagents import CodeAgent, InferenceClientModel, tool

MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [ ]:
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest rated catering service in Gothham City.

    Args:
        query (str): The user's query about catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    best_service = max(services, key=services.get)
    return best_service

In [ ]:
agent = CodeAgent(tools=[catering_service_tool], model=InferenceClientModel(MODEL))

# Run the agent to find the best catering service
result = agent.run(
    "Can you give me the name of the highest-rated catering service in Gotham City?"
)

print(result)   # Output: Gotham Catering Co.

## Defining a tool as a python class
This approach involves creating a subclass of Tool. For complex tools, we can implement a class instead of a Python function. The class wraps the function with metadata that helps the LLM understand how to use it effectively. In this class, we define:

* name: The tool’s name.
* description: A description used to populate the agent’s system prompt.
* inputs: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
* output_type: Specifies the expected output type.
* forward: The method containing the inference logic to execute.

In [ ]:
from smolagents import Tool

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = "This tool suggests creative superhero-themed party ideas based on a category. It returns a unique party theme idea."
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham')."
        }
    }
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")

agent = CodeAgent(tools=[SuperheroPartyThemeTool()], model=InferenceClientModel(MODEL))
result = agent.run("Suggest a unique party theme for a 'villain masquerade' superhero party.")
print(result)  # Output: Gotham Rogues' Ball: A mysterious masquerade where guests

In [ ]:
@tool
def convert_km_to_miles(km: float) -> float:
    """
    Convert kilometers to miles.
    Args:
        km (float): Distance in kilometers.
    """
    return km * 0.621371
agent = CodeAgent(tools=[convert_km_to_miles], model=InferenceClientModel(MODEL))
result = agent.run("Convert 10 kilometers to miles.")   
print(result)  # Output: 6.21371

In [ ]:
# Note: Use environment variable or prompt for token to avoid exposing it in code
# convert_km_to_miles.push_to_hub("cfcooney", token=os.getenv("HF_TOKEN"))
# Or prompt for token: 
# from getpass import getpass
# token = getpass("Enter your HuggingFace token: ")
# convert_km_to_miles.push_to_hub("cfcooney", token=token)

print("Token removed for security. Use environment variable or secure input method.")

### Importing tools from the hub

In [ ]:
from smolagents import load_tool

image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=InferenceClientModel(MODEL)
)

agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")

In [ ]:
agent.run("Generate an image of a cyclist on a road bike climbing up a steep alpine mountain. The image should not be photorealistic but rather in the style of a detailed digital painting.")

### Importing from a Huggingface Space as a tool

In [ ]:
from smolagents import CodeAgent, InferenceClientModel, Tool

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)

model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[image_generation_tool], model=model)

agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}
)

### Importing a Langchain Tool

In [ ]:
# Fix for newer LangChain versions - load_tools has moved
try:
    # Try the old import first
    from langchain.agents import load_tools
    print("✅ Using langchain.agents.load_tools (older version)")
except ImportError:
    try:
        # Try the new import path
        from langchain_community.agent_toolkits.load_tools import load_tools
        print("✅ Using langchain_community.agent_toolkits.load_tools (newer version)")
    except ImportError:
        try:
            # Another possible location
            from langchain_community.tools import load_tools
            print("✅ Using langchain_community.tools.load_tools")
        except ImportError:
            print("❌ Could not import load_tools from any location")
            print("You may need to install: pip install langchain-community")

from smolagents import CodeAgent, InferenceClientModel, Tool

In [ ]:
import os
#os.environ["SERPAPI_API_KEY"] =
# Load the LangChain serpapi tool
lc_tool = load_tools(['serpapi'])[0]

# Wrap it manually for smolagents
def serpapi_search(query: str):
    """Perform a search using SerpAPI via LangChain."""
    return lc_tool.run(query)

search_tool = Tool(
    name="serpapi_search",
    description="Search the web using SerpAPI",
    func=serpapi_search
)

agent = CodeAgent(tools=[search_tool], model=MODEL)

agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")

## Importing tools from any MCP Server

**Model Context Protocol (MCP)** allows you to connect to various external services and tools. This example attempts to connect to a PubMed medical research server.

### Requirements:
- `mcp` package: `pip install mcp`
- `pywin32` package (Windows): `pip install pywin32` 
- **`uv` tool** for Python package management
- Network access to MCP servers

### Installing `uv` (Required for MCP):

**Windows (PowerShell):**
```powershell
curl -LsSf https://astral.sh/uv/install.ps1 | powershell
```

**Alternative (via pip):**
```bash
pip install uv
```

**After installation:**
- Restart your terminal/VS Code
- Verify with: `uv --version`

### Common Issues:
- **"Couldn't connect after 30 seconds"** → `uv` not installed
- **"uvx command not found"** → Add `uv` to PATH or restart terminal
- **Connection timeout** → Network/firewall issues

**Note:** If MCP connection fails, the code will fallback to a comprehensive mock tool for demonstration.

In [ ]:
import os
import subprocess
import sys
from smolagents import CodeAgent, InferenceClientModel, tool

# Define the fallback tool first (outside try/except to avoid indentation issues)
@tool
def enhanced_medical_search(query: str) -> str:
    """Enhanced mock medical search with more detailed information.
    
    Args:
        query: Medical query or condition to search for
    """
    medical_db = {
        "hangover": {
            "condition": "Hangover (Alcohol withdrawal symptoms)",
            "remedies": [
                "Hydration: 8-10 glasses of water throughout the day",
                "Electrolyte replacement: Sports drinks, coconut water, or oral rehydration solutions",
                "Rest: 7-9 hours of sleep to aid recovery",
                "Nutrition: Light, easily digestible foods (toast, bananas, crackers)",
                "Pain relief: Ibuprofen or aspirin (avoid acetaminophen with alcohol)",
                "Vitamin B complex: May help with alcohol metabolism",
                "Ginger: For nausea relief (tea or supplements)"
            ],
            "avoid": [
                "More alcohol ('hair of the dog')",
                "Heavy, greasy foods",
                "Excessive caffeine",
                "Acetaminophen (Tylenol) - can damage liver when combined with alcohol"
            ],
            "when_to_seek_help": "If symptoms persist >24 hours, severe dehydration, or concerning symptoms"
        },
        "headache": {
            "condition": "Headache",
            "remedies": ["Rest in dark room", "Hydration", "Cold/warm compress", "OTC pain relievers"],
            "avoid": ["Bright lights", "Loud noises", "Dehydration"]
        }
    }
    
    # Search for relevant condition
    for condition, info in medical_db.items():
        if condition.lower() in query.lower():
            result = f"""
🏥 Medical Information: {info['condition']}

💊 Recommended Remedies:
{chr(10).join(f"• {remedy}" for remedy in info['remedies'])}

⚠️ Things to Avoid:
{chr(10).join(f"• {avoid}" for avoid in info['avoid'])}

🚨 Seek Medical Help: {info.get('when_to_seek_help', 'If symptoms worsen or persist')}

⚖️ Disclaimer: This is educational information only. Always consult healthcare professionals for medical advice.
            """
            return result.strip()
    
    return f"Mock medical search results for: {query}\n\nSuggestion: Try searching for 'hangover', 'headache', or other common conditions."

def check_uv_installation():
    """Check if uv/uvx is installed and available"""
    try:
        result = subprocess.run(['uvx', '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"✅ uvx is available: {result.stdout.strip()}")
            return True
    except (subprocess.TimeoutExpired, FileNotFoundError):
        pass
    
    try:
        result = subprocess.run(['uv', '--version'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            print(f"✅ uv is available: {result.stdout.strip()}")
            return True
    except (subprocess.TimeoutExpired, FileNotFoundError):
        pass
    
    print("❌ Neither 'uv' nor 'uvx' commands are available")
    return False

# Try MCP connection
try:
    from smolagents import ToolCollection
    from mcp import StdioServerParameters
    
    print("✅ MCP imports successful")
    
    model = InferenceClientModel("Qwen/Qwen2.5-Coder-32B-Instruct")
    
    # Check if uv is available
    if not check_uv_installation():
        print("\n💡 To fix this, install uv:")
        print("Windows: curl -LsSf https://astral.sh/uv/install.ps1 | powershell")
        print("Or: pip install uv")
        print("Then restart your terminal/notebook")
        raise Exception("uv not installed")
    
    print(f"\n🔍 Attempting to connect to PubMed MCP server...")
    
    # Try to connect to MCP server with shorter timeout
    server_parameters = StdioServerParameters(
        command="uvx",
        args=["--quiet", "pubmedmcp@0.1.3"],
        env={"UV_PYTHON": "3.12", **os.environ},
    )
    
    # Reduce timeout to 10 seconds instead of 30
    with ToolCollection.from_mcp(server_parameters, trust_remote_code=True, timeout=10) as tool_collection:
        print(f"✅ Connected! Found {len(tool_collection.tools)} tools")
        
        # List available tools
        print("Available tools:")
        for tool_item in tool_collection.tools:
            print(f"  - {tool_item.name}: {tool_item.description}")
        
        agent = CodeAgent(tools=[*tool_collection.tools], model=model, add_base_tools=True)
        result = agent.run("Please find a remedy for hangover.")
        print(f"🎯 MCP Result: {result}")
        
except Exception as e:
    print(f"\n❌ MCP connection failed: {e}")
    print("\n📋 Diagnosis:")
    print("✅ uvx tool is installed and working")
    print("❌ PubMed MCP server connection timeout")
    print("💡 This could be due to:")
    print("   - Network/firewall blocking the connection")
    print("   - PubMed MCP server being unavailable")
    print("   - Corporate proxy settings")
    
    print("\n🔄 Using enhanced fallback medical tool...")
    agent = CodeAgent(tools=[enhanced_medical_search], model=model, add_base_tools=True)
    result = agent.run("Please find a comprehensive remedy for hangover including what to avoid.")
    print(f"🎯 Enhanced Fallback result: {result}")

print("\n" + "="*60)
print("📋 Summary: MCP Server Integration Status")
print("="*60)
print("✅ MCP framework is properly set up")
print("✅ uvx tool is installed and working") 
print("❌ PubMed server connection failed (likely network/server issue)")
print("🔄 Fallback provides comprehensive medical information")
print("💡 Your MCP setup is correct - try different servers or check network")